In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
df = pd.read_csv('data_final.csv', sep=';')
data_dict = pd.read_excel("Dictionnaire_final.xlsx")


# Drop superfluous columns
df = df.drop(columns=["Unnamed: 59", "Unnamed: 60"])
# Drop english columns
df = df.drop(columns=["street_1", "street_2"])

In [10]:
df.head()

,int_no,x,y,rue_1,rue_2,street_1,street_2,date_,all_pedest,pi,...,any_exclus,commercial,curb_exten,all_red_an,new_half_r,distdt,ln_distdt,traffic_10000,ped_100,borough
0,40,296397.31250,5037515.0,Côte-Saint-Antoine,Hampton/Sherbrooke,C¶te-Saint-Antoine,Hampton/Sherbrooke,27/02/2009,0,0.0,...,0,0,0,0,1,3932.076904,8.276923,0.260383,0.0,C¶te-des-Neiges-Notre-Dame-de-Graces
1,263,300293.31250,5038406.5,Centre,Wellington,Centre,Wellington,20/11/2007,0,0.0,...,0,1,0,0,1,2097.060059,7.648292,0.781605,0.0,Sud-Ouest
2,1107,301317.09375,5039803.0,Pierre-Dupuy,Habitats 67,Pierre-Dupuy,Habitats 67,09/09/2009,0,0.0,...,0,2,1,0,0,2112.164062,7.655468,0.889667,0.0,Ville-Marie
3,1364,299985.87500,5037138.5,Charlevoix,Wellington,Charlevoix,Wellington,10/12/2007,0,0.0,...,0,0,0,0,0,3172.770020,8.062361,1.093276,0.0,Sud-Ouest
4,225,297859.78125,5036983.0,Courcelle,Notre-Dame,Courcelle,Notre-Dame,15/11/2006,0,0.0,...,0,0,0,0,0,3525.406006,8.167751,1.047930,0.0,Sud-Ouest


In [14]:
pd.set_option('display.max_colwidth', None)
data_dict

,NOM,DESCRIPTION
0,int_no,Intersection ID
1,x,latitude
2,y,longitude
3,rue_1,first street of the intersection
4,rue_2,second street of the intersection
5,street_1,first street of the intersection
6,street_2,second street of the intersection
7,date,date where safety measures where recorded
8,all_pedest,presence of all pedestrian phase (exclusive pedestrian phase)
9,pi,average annual daily flow for pedestrian at the intersection


In [15]:
# Duplicates?
print(df["int_no"].value_counts().sort_values().tail(5))
df[df["int_no"] == 9151]

,int_no,x,y,rue_1,rue_2,street_1,street_2,date_,all_pedest,pi,...,curb_exten,all_red_an,new_half_r,distdt,ln_distdt,traffic_10000,ped_100,borough,Unnamed: 59,Unnamed: 60
731,9151,297769.09375,5050923.5,Langelier,P,Langelier,P,15/07/2009,0,894.321106,...,0,0,0,10801.348633,9.287426,2.188579,8.943211,St-LÚonard,8.943211,St-LÚonard
756,9151,298515.93750,5049434.5,Jean-Talon,Mainville,Jean-Talon,Mainville,21/09/2009,0,1124.394653,...,0,0,0,9239.489258,9.131242,1.292356,11.243946,St-LÚonard,NaN,NaN
